In [1]:
import os
import sys
sys.path.append('../modules')

import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = InteractiveSession(config=config)

from srgan_d import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
r = 2
input_dir = f'../data/train/{r}/low/'
tgt_dir = f'../data/train/{r}/high/'

input_dir_val = f'../data/val/{r}/low/'
tgt_dir_val = f'../data/val/{r}/high/'


test_file = 'UBMk30rjy0o_17675_42.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_dir = '../data/out/'

model_dir = '../models/'
vgg_name = 'vgg16_notop.hdf5'
vgg_path = model_dir + vgg_name

input_shape = (67, 120, 3)
tgt_shape = (134, 240, 3)

batch_size = 8

In [3]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

../modules/srgan_d.py:111: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
../modules/srgan_d.py:153: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
srgan.generator.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 67, 120, 3)   0                                            
__________________________________________________________________________________________________
G_Head_conv0 (Conv2D)           (None, 67, 120, 128) 3584        input_1[0][0]                    
__________________________________________________________________________________________________
G_Head_norm0 (BatchNormalizatio (None, 67, 120, 128) 512         G_Head_conv0[0][0]               
__________________________________________________________________________________________________
G_Head_act0 (LeakyReLU)         (None, 67, 120, 128) 0           G_Head_norm0[0][0]               
____________________________________________________________________________________________

In [5]:
for e in range(100001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = (cv2.imread(test_low_path) / 255.0)[np.newaxis, :, :, :]
        test_img_high = (cv2.imread(test_high_path) / 255.0)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = (test_img_sr * 255).astype('u1')[0]
        test_out_path = test_out_dir + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:1.006349802017212, d_loss_fake:0.5265344977378845, vgg_loss:4.845170021057129
epoch 100
d_loss_real:0.6426376700401306, d_loss_fake:0.7629549503326416, vgg_loss:0.21590431034564972
epoch 200
d_loss_real:0.7431495189666748, d_loss_fake:0.6693935394287109, vgg_loss:0.18525491654872894
epoch 300
d_loss_real:0.707329511642456, d_loss_fake:0.6659530401229858, vgg_loss:0.09708874672651291
epoch 400
d_loss_real:0.6692461371421814, d_loss_fake:0.7082675695419312, vgg_loss:0.19153687357902527
epoch 500
d_loss_real:0.6743482351303101, d_loss_fake:0.7110433578491211, vgg_loss:0.15467534959316254
epoch 600
d_loss_real:0.6826440095901489, d_loss_fake:0.7008882164955139, vgg_loss:0.15681545436382294
epoch 700
d_loss_real:0.6936624050140381, d_loss_fake:0.6856681704521179, vgg_loss:0.1446492075920105
epoch 800
d_loss_real:0.6997416019439697, d_loss_fake:0.6831711530685425, vgg_loss:0.13560032844543457
epoch 900
d_loss_real:0.6760739088058472, d_loss_fake:0.6843699216842651, vgg_lo

epoch 8000
d_loss_real:1.3770344257354736, d_loss_fake:0.12602929770946503, vgg_loss:0.13105221092700958
epoch 8100
d_loss_real:1.5115493535995483, d_loss_fake:0.5571421384811401, vgg_loss:0.10648508369922638
epoch 8200
d_loss_real:0.8446648716926575, d_loss_fake:0.7345095872879028, vgg_loss:0.06278116255998611
epoch 8300
d_loss_real:0.6616392731666565, d_loss_fake:0.20459559559822083, vgg_loss:0.15779882669448853
epoch 8400
d_loss_real:2.2095353603363037, d_loss_fake:0.23778878152370453, vgg_loss:0.18414853513240814
epoch 8500
d_loss_real:1.3720591068267822, d_loss_fake:0.6413388252258301, vgg_loss:0.10739965736865997
epoch 8600
d_loss_real:0.8329222798347473, d_loss_fake:0.2346740961074829, vgg_loss:0.10991036891937256
epoch 8700
d_loss_real:1.169644832611084, d_loss_fake:0.3890535235404968, vgg_loss:0.0850239172577858
epoch 8800
d_loss_real:1.9903614521026611, d_loss_fake:0.15040339529514313, vgg_loss:0.11754606664180756
epoch 8900
d_loss_real:1.8068643808364868, d_loss_fake:0.42554

epoch 15900
d_loss_real:0.9263768792152405, d_loss_fake:0.24007606506347656, vgg_loss:0.06959626823663712
epoch 16000
d_loss_real:1.192168116569519, d_loss_fake:0.030985258519649506, vgg_loss:0.19611679017543793
epoch 16100
d_loss_real:1.5693796873092651, d_loss_fake:0.2085348665714264, vgg_loss:0.08353852480649948
epoch 16200
d_loss_real:0.7707291841506958, d_loss_fake:0.0957101434469223, vgg_loss:0.08184859901666641
epoch 16300
d_loss_real:1.3266172409057617, d_loss_fake:0.18073424696922302, vgg_loss:0.15116094052791595
epoch 16400
d_loss_real:1.655576467514038, d_loss_fake:0.28186851739883423, vgg_loss:0.068427175283432
epoch 16500
d_loss_real:2.569387435913086, d_loss_fake:0.20757019519805908, vgg_loss:0.1475887894630432
epoch 16600
d_loss_real:2.248866081237793, d_loss_fake:0.11154419928789139, vgg_loss:0.18137218058109283
epoch 16700
d_loss_real:0.6196715831756592, d_loss_fake:0.16710561513900757, vgg_loss:0.06243783235549927
epoch 16800
d_loss_real:1.3250138759613037, d_loss_fak

epoch 23800
d_loss_real:1.2801835536956787, d_loss_fake:1.2967060804367065, vgg_loss:0.09703561663627625
epoch 23900
d_loss_real:2.0117058753967285, d_loss_fake:0.13277451694011688, vgg_loss:0.04575296863913536
epoch 24000
d_loss_real:0.5566569566726685, d_loss_fake:0.3112122118473053, vgg_loss:0.1472424417734146
epoch 24100
d_loss_real:1.4728310108184814, d_loss_fake:0.11823468655347824, vgg_loss:0.1416768878698349
epoch 24200
d_loss_real:0.4577425718307495, d_loss_fake:0.021577224135398865, vgg_loss:0.1142791211605072
epoch 24300
d_loss_real:2.8180489540100098, d_loss_fake:0.5764825344085693, vgg_loss:0.06578624993562698
epoch 24400
d_loss_real:1.3272202014923096, d_loss_fake:0.0193512924015522, vgg_loss:0.10262890160083771
epoch 24500
d_loss_real:0.6993992924690247, d_loss_fake:0.09328802675008774, vgg_loss:0.24865354597568512
epoch 24600
d_loss_real:2.2297773361206055, d_loss_fake:0.03671257942914963, vgg_loss:0.06680969893932343
epoch 24700
d_loss_real:1.3455572128295898, d_loss_f

epoch 31600
d_loss_real:0.9549542665481567, d_loss_fake:0.09869124740362167, vgg_loss:0.09259764850139618
epoch 31700
d_loss_real:0.5525010824203491, d_loss_fake:0.14577248692512512, vgg_loss:0.07838770747184753
epoch 31800
d_loss_real:0.058261968195438385, d_loss_fake:0.2032814770936966, vgg_loss:0.12058961391448975
epoch 31900
d_loss_real:1.1929504871368408, d_loss_fake:0.03745678439736366, vgg_loss:0.045339617878198624
epoch 32000
d_loss_real:0.36561644077301025, d_loss_fake:0.5882214307785034, vgg_loss:0.40063345432281494
epoch 32100
d_loss_real:1.1355996131896973, d_loss_fake:0.08401195704936981, vgg_loss:0.1419093757867813
epoch 32200
d_loss_real:3.39016056060791, d_loss_fake:0.01809736154973507, vgg_loss:0.14354172348976135
epoch 32300
d_loss_real:0.6867033243179321, d_loss_fake:0.9605715274810791, vgg_loss:0.029336119070649147
epoch 32400
d_loss_real:0.4186372756958008, d_loss_fake:0.007966699078679085, vgg_loss:0.14628079533576965
epoch 32500
d_loss_real:0.3167703151702881, d_

epoch 39400
d_loss_real:0.45257097482681274, d_loss_fake:0.32503288984298706, vgg_loss:0.10279269516468048
epoch 39500
d_loss_real:1.6131843328475952, d_loss_fake:0.16622896492481232, vgg_loss:0.03768732771277428
epoch 39600
d_loss_real:0.2426058053970337, d_loss_fake:0.038556575775146484, vgg_loss:0.06244068965315819
epoch 39700
d_loss_real:1.448559284210205, d_loss_fake:0.3158051371574402, vgg_loss:0.05732082575559616
epoch 39800
d_loss_real:0.5025941133499146, d_loss_fake:0.010245992802083492, vgg_loss:0.10815785825252533
epoch 39900
d_loss_real:0.48815348744392395, d_loss_fake:0.09948989748954773, vgg_loss:0.08401595801115036
epoch 40000
d_loss_real:0.48605212569236755, d_loss_fake:0.03775114566087723, vgg_loss:0.06819095462560654
epoch 40100
d_loss_real:0.8889614939689636, d_loss_fake:0.11946664750576019, vgg_loss:0.1004137247800827
epoch 40200
d_loss_real:1.6603273153305054, d_loss_fake:0.14927247166633606, vgg_loss:0.06823684275150299
epoch 40300
d_loss_real:1.5490145683288574, 

KeyboardInterrupt: 

In [6]:
out_model_name = '20200522_generator.h5'
srgan.generator.save(model_dir + out_model_name, include_optimizer=False)